In [37]:
## Read PDF files
import nest_asyncio

nest_asyncio.apply()
#GET LLAMA_CLOUD_API_KEY
import os
from llama_parse import LlamaParse

api_key = os.getenv("LLAMA_CLOUD_API_KEY")
print(api_key)

parser = LlamaParse(
    api_key= "api_key",  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown",  # "markdown" and "text" are available
    num_workers=4,  # if multiple files passed, split in `num_workers` API calls
    verbose=True,
    language="fr",  # Optionally you can define a language, default=en
)
# sync
documents = parser.load_data("./docs/CHARTE_QUALITE_PRODUIT.pdf")
print(documents)


None
Started parsing the file under job_id 2059df9d-af51-4fa0-95d1-3517927c4744
[Document(id_='ef1a9119-89ad-4da7-852f-62d8bc693963', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='# Codification :\n\n# Coupe de Pates CHARTE PRODUITS COUP DE PATES CDP_QUA_CHART_01\n\nVersion : 5\n\nDate d’application : 13/12/2023\n\nDate Mises à jour\n\n|19/12/2014|Création|\n|---|---|\n|12/12/2019|Insertion des additifs interdits et à éviter|\n|13/05/2022|Revue des exigences reçues et annexes|\n|30/03/2023|Revue des annexes I et II. Fréquence de mise à jour CDC|\n|13/12/2023|Ajout d’une exigence de certification sur le cacao Revue des exigences de certification de l’huile de palme|\n\n# Table des matières\n\n1. Exigence reçue\n2. Produits soumis à certification ou allégations\n3. 1. Produits « sans gluten »\n2. Produits issus de l’agriculture biologique\n\nExigences générales relatives au fournisseur\n4. Exigences relatives aux site

In [47]:
print(f"DOC LENGTH : {len(documents[0].text)} characters")
print(documents[0].text[3000:4000])

DOC LENGTH : 27448 characters
ion OGM|INTERDIT|INTERDIT|INTERDIT|
|Traitement par Ionisation|INTERDIT|INTERDIT|INTERDIT|
|Colorants azoïques (E102, E104, E110,E122, E124, E129)|INTERDIT|INTERDIT|INTERDIT|
|Nanoparticules (E170, E171, E172,E174, E152, E341, E551 et E552)|INTERDIT|INTERDIT|INTERDIT|
|Glutamates et exhausteurs de goût|INTERDIT|INTERDIT|INTERDIT|
|Œufs de poules élevées en cage|INTERDIT|INTERDIT|INTERDIT|
|Matières grasses partiellementhydrogénées|INTERDIT|INTERDIT|INTERDIT|
|Acides gras trans non naturellementprésents|INTERDIT|INTERDIT|INTERDIT|
|Edulcorants de synthèse|INTERDIT|INTERDIT|INTERDIT|
|Viande Séparée Mécaniquement - VSM|INTERDIT|INTERDIT|INTERDIT|
|Cacao non certifié durable|INTERDIT| |* interdit pour tous les NPD et plan action pour remplacer le cacao non certifié dans l'existant.|
|Gélatine porcine|INTERDIT|INTERDIT|INTERDIT|
|Gélatine animale – (autre que porcine)|A EVITER|INTERDIT|INTERDIT|
| |tolérance dans les pâtisseries|tolérance dans les pâtisseries|

In [48]:
# function to Delete elements of more then 30 char that repeats itself in the text
def remove_headers_footers(ocr_text, min_length=40, min_repetitions=3):
    segments = ocr_text.split("\n")  # Split OCR text into segments (assuming newline-separated)
    repeated_segments = {}  # Dictionary to store repeated segments and their counts

    # Count repetitions of each segment
    for segment in segments:
        if len(segment) >= min_length:
            if segment in repeated_segments:
                repeated_segments[segment] += 1
            else:
                repeated_segments[segment] = 1

    # Identify segments to remove based on repetition count
    segments_to_remove = []
    for segment, count in repeated_segments.items():
        if count >= min_repetitions:
            segments_to_remove.append(segment)

    print(segments_to_remove)
    # Remove identified segments from OCR text
    cleaned_text = ocr_text
    for segment in segments_to_remove:
        cleaned_text = cleaned_text.replace(segment, "")

    return cleaned_text.strip()  # Strip leading and trailing whitespace





In [64]:
cleaned_text = remove_headers_footers(documents[0].text, min_length=10, min_repetitions=3)

['# Codification :', 'Version : 5', 'Date d’application : 13/12/2023', 'CDP_QUA_CHART_01', '# CHARTE PRODUITS COUP DE PATES Version : 5', '# CHARTE PRODUITS COUP DE PATES', '# Date d’application :', '13/12/2023', '|---|---|---|---|', 'Date d’application: 13/12/2023', 'Date d’application :', 'CHARTE PRODUITS COUP DE PATES Version : 5', '|---|---|---|', '|Codification :|CDP_QUA_CHART_01|', '|Date d’application :| |', '|CHARTE PRODUITS COUP DE PATES|Version : 5|']


In [65]:
import re
cleaned_text = re.sub(r"^Page.*$", "", cleaned_text, flags=re.MULTILINE)#delete the entire line that start with "page .."
cleaned_text =  re.sub(r"\n+", "\n", cleaned_text) #delete multiple new lines
print(f"Cleaned text LENGTH : {len(cleaned_text)} characters")
print(cleaned_text[3500:5000])

Cleaned text LENGTH : 24189 characters
ITER|INTERDIT|INTERDIT|
| |tolérance dans les pâtisseries|tolérance dans les pâtisseries| |
---
| |INTERDIT|A EVITER|A EVITER|INTERDIT|
---|
|Huile de palme + palmiste non RSPO|* interdit pour tous les NPD et plan action pour retirer dans l'existant - (tolérée dans supports d’additifs) - En aucun cas, l’huilede palme non RSPO ne pourra être substituée par de l’huile de coprah ou coco.| | | |
|Huile de palme + palmiste RSPO| |A EVITER|A EVITER|INTERDIT|
|Arômes artificiels| | |tolérance dans lespâtisseries|INTERDIT|
| | | |*sauf arômes de fumée et vanilline| |
---
|Colorants artificiels|A EVITER|A EVITER|INTERDIT|
| |Interdit dans les produits salés| | |
Ingrédients controversés (cf.Annexe IV)
A EVITER
A EVITER
INTERDIT
|Additifs rouges (cf. Annexe I)|A EVITER|INTERDIT|INTERDIT|
| |hors nitrites et polyphosphates| |hors nitrites dans les produits de salaison|
Additifs Oranges (cf. Annexe II)
A EVITER
A EVITER
A EVITER
|Nitrites (E250 à E252)|A EVIT

In [ ]:
# Ask an llm to rewrite the text in a more readable & understandable way